# Adding constraints derived from a structure

In [1]:
# You have to rerun this cell each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Notebook is set for PyRosetta use in Colab.  Have fun!


Now cd into the correct directory

In [2]:
# cd into the correct directory
!cd google_drive/MyDrive/CodeSchool2024/

We initialize PyRosetta without any command line options.

In [3]:
from pyrosetta import *
init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.19+release.a34b73c40fe9c61558d566d6a63f803cfb15a4fc 2024-05-02T16:22:03] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r381 2024.19+release.a34b73c40f a34b

Initiliazing pose and score function.

In [4]:
# initialize the pose with the 1ubq.pbd, initialize the full atom ref2015 score function, and score your pose

# YOUR CODE HERE (initialize pose)
pose = pyrosetta.pose_from_file("google_drive/MyDrive/CodeSchool2024/inputs/1ubq.pdb")

# YOUR CODE HERE (get score function)
sfxn = get_score_function(True)

# YOUR CODE HERE (score pose)
sfxn(pose)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.32329 seconds.
core.import_pose.import_pose: File 'google_drive/MyDrive/CodeSchool2024/inputs/1ubq.pdb' automatically determined to be of type PDB
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/sco

32.67775371933962

Let's prime the score function with a constraint score term. Let's also print out the score before we've added any constraints to the pose so we can refer back to it later.

In [5]:
from pyrosetta.rosetta.core.scoring import *

# set the weight of the atom_pair_constraint to be 1
# YOUR CODE HERE
sfxn.set_weight(atom_pair_constraint, 1)
# use sfxn.show to show the score function breakdown of the pose

# YOUR CODE HERE
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

What is the weight of the atom_pair_constraint? what is the raw score of the atom_pair_constraint? Why?

YOUR ANSWER HERE:
it is 1. the raw score is 0 because we haven't actually added the weight yet.

To define where the constraints need to be applied, we will need to use residue selectors. Let's use a `LayerSelector` to select the surface residues.

In [9]:
from pyrosetta.rosetta.core.select.residue_selector import *
surface_res = LayerSelector()

# use surface_res.set_layers? to determine how to select only the surface residues

# YOUR CODE HERE (select the surface residues
surface_res.set_layers(False, False, True)

core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=false boundary=false surface=true in LayerSelector.


To check which residues are selected on the pose, we can apply it. `1` means that the residue has been selected, `0` means that it has not.

What residues have been selected?

YOUR ANSWER HERE: only the surface residues have been selected.

In [10]:
surface_res.apply(pose)

vector1_bool[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]

Now let's create atom pair constraints. For this, we use `AtomPairConstraintGenerator`. **By default, it uses the Sum Of Gaussians function to score the distances between atoms.** The "sum" part is not really used, because there is just one gaussian. This gaussian turns a distance into a probability, `prob`, and the energy is taken as the `-ln(prob)`.   We will pass the residue selector to the generator, set the maximum distance at under which it will create atom pair constraints (5.0 Ang), the standard deviation of the gaussians, and set the constraints to only be generated between pairs of C-alpha atoms.

In [12]:
from pyrosetta.rosetta.protocols.constraint_generator import *
apcg = AtomPairConstraintGenerator()

# use the set_residue_selector method of apcg to set the residue selector to be our surface_res selector above
# YOUR CODE HERE
apcg.set_residue_selector(surface_res)


# use the set_max_distance method of apcg to set the max distance to be 5.0
# YOUR CODE HERE
apcg.set_max_distance(5.0)

# use set_sd to set the standard deviation of the gaussian to be 1
# YOUR CODE HERE
apcg.set_sd(1)

# we will only look at atom pair constraints for the carbon alpha atoms. we do this by using:
apcg.set_ca_only(True)

While we could directly apply the `AtomPairConstraintGenerator` to the pose, the recommended method is to add it to an `AddConstraints` object. Multiple constraint generators can be added to this. We then apply all the constraints.

In [13]:
add_csts = AddConstraints()
add_csts.add_generator(apcg)
add_csts.apply(pose)

protocols.constraint_generator.AddConstraints: Adding 9 constraints generated by ConstraintGenerator named unnamed_constraint_generator


Let's check if the constraints were applied.

In [14]:
# use sfxn.show to see if the constraints were applied

# YOUR CODE HERE
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

What is the raw score and weighted score of the atom_pair_constraint?

YOUR ANSWER HERE: raw score and the weighted score is -129.503

The sum of gaussians function acts more like a reward. A large neagtive number indicates that all 9 constraints were satisfied. What happens when we disrupt some surface interactions in a perturbed structure?

In [15]:
pose_perturbed = pose.clone()  # copy the conformation and the constraints

# set the phi and psi angles of residue 25 to be -130, and 145, respectively

# YOUR CODE HERE
pose.set_phi(25, -130)
pose.set_psi(25, 145)

In [16]:
# show the score breakdown

# YOUR CODE HERE
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -575.051    -575.051
 fa_rep                       0.550   55367.773   30452.275
 fa_sol                       1.000     413.031     413.031
 fa_intra_rep                 0.005     355.301       1.777
 fa_intra_sol_xover4          1.000      16.485      16.485
 lk_ball_wtd                  1.000     -16.198     -16.198
 fa_elec                      1.000     -84.545     -84.545
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -15.276     -15.276
 hbond_lr_bb                  1.000     -13.621     -13.621
 hbond_bb_sc                  1.000      -7.087      -7.087
 hbond_sc                     1.000      -2.165      -2.165
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

What is the atom_pair_constraint raw score now?

YOUR ANSWER HERE:

The reward is much smaller owing to the perturbation of the pose.